
### A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts. 
### For that we need to build a multiple linear regression model for the prediction of car prices.

# 1. Reading and understanding data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',50)
%matplotlib inline

In [2]:
# Check the head of the dataset
df=pd.read_csv("CarPrice_Assignment.csv")
df.head()

FileNotFoundError: [Errno 2] File b'CarPrice_Assignment.csv' does not exist: b'CarPrice_Assignment.csv'

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df=df.drop(['car_ID'],axis=1) #dropping car id column as it is not required
df.head()

In [ ]:
df.isnull().sum() # find the sum of null values in all the columns

insight: There are no null values in any columns

### Finding Car company name from CarName column

In [ ]:
# splitting car company name from CarName column
df['CarName']=df['CarName'].str.split(' ',expand=True)
df.head()

In [ ]:
df['CarName'].unique() #finding unique carname

There are some company name which is spelled incorrectly<br>
      mazda = maxda <br>
      Nissan = nissan <br>
      porsche = porcshce <br>
      toyota = toyouta <br>
      vokswagen = volkswagen = vw <br>

In [ ]:
#replacing incorrect company name by correct one
df['CarName']=df['CarName'].replace({'maxda':'mazda','nissan':'Nissan','porcshce':'porsche','toyouta':'toyota','vw':'volkswagen','vokswagen':'volkswagen'})
df.head()

In [ ]:
df['CarName'].unique()

### Categorizing symboling column<br>
Here Symboling is the assigned insurance risk rating,value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.
Let's make the category clear by classifying based on risk:

-ve symboling as safe<br>
0, 1 as moderate<br>
2,3 as risky<br>

In [ ]:
#using map function to assign category
df['symboling'] = df['symboling'].map({-2:'safe',-1:'safe',0:'moderate',1:'moderate',2:'risky',3:'risky'})

In [ ]:
df.head()

In [ ]:
#finding the list of all the categorical values
categorical_variables=list(df.columns[df.dtypes=='object'])
categorical_variables

### EDA for detailed analysis

In [ ]:
#Plots for all the categorical values
plt.figure(figsize=(20,20))
plt.subplot(5,2,1)
sns.countplot('symboling',data=df)
plt.subplot(5,2,2)
sns.countplot('fueltype',data=df)
plt.subplot(5,2,3)
sns.countplot('aspiration',data=df)
plt.subplot(5,2,4)
sns.countplot('doornumber',data=df)
plt.subplot(5,2,5)
sns.countplot('carbody',data=df)
plt.subplot(5,2,6)
sns.countplot('drivewheel',data=df)
plt.subplot(5,2,7)
sns.countplot('enginelocation',data=df)
plt.subplot(5,2,8)
sns.countplot('enginetype',data=df)
plt.subplot(5,2,9)
sns.countplot('cylindernumber',data=df)
plt.subplot(5,2,10)
sns.countplot('fuelsystem',data=df)
plt.show()

insights : We can identify some of the car features that are predominant in the US Automobile Market.These features are:<br>

-symboling: moderate (0,1)<br>
-Carbody: Sedan<br>
-fueltype: gas<br>
-aspiration: standard<br>
-doornumbers: four<br>
-drivewheel: forward<br>
-engine location: front<br>
-engine type: ohc<br>
-cylinderNumber: four<br>
-fuelSystem: mpfi<br>

In [ ]:
# plotting carname to find which company has maximum price
plt.figure(figsize=(20,10))
sns.countplot('CarName',data=df)
plt.show()

insight:From above graph it is clearly visible that Toyota has dominant market followed by Nissan and mazda.

In [ ]:
# Heat map to find correlation between diffrent numeric variables
plt.figure(figsize = (20, 20))
ax=sns.heatmap(df.corr(),annot=True,cmap="rainbow")
x,y=ax.get_ylim()
ax=ax.set_ylim(x+0.5,y-0.5)
plt.show()

insight : From above plot we can clearly see there are many varaibles which have good correlation with price like enginesize,horsepower,curbweight,carwidth.<br>
we can also observe some multicollinearity among varaibles like enginesize and horsepower,wheelbase and carlength,citympg and highwaympg etc

# 2. Visualising the data

### paiplot of all numeric values

In [ ]:
numeric_values=['wheelbase','carlength','carwidth','carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg','price']

In [ ]:
plt.figure(figsize=(50,80))
g=sns.pairplot(df,x_vars=["carlength","wheelbase","carwidth","carheight","curbweight","enginesize","boreratio",],y_vars=["price"])
plt.show()

In [ ]:
plt.figure(figsize=(50,80))
g=sns.pairplot(df,x_vars=["stroke","compressionratio","horsepower","peakrpm","citympg","highwaympg"],y_vars=["price"])
plt.show()

We see enginesize, horsepower and compression ratio variables to have outliers .<br>
Enginesize,curbweight,horsepower has high positive correlation with price whereas citympg,highwaympg is negatively correlated with price

### outliers treatment

In [ ]:
print(df[['horsepower','curbweight','enginesize']].quantile([0.01, .96])) #fining the percentile values
print(df[['compressionratio']].quantile([0.01, .90]))#fining the percentile values

In [ ]:
# treating Outilers in price of cars
df['horsepower'][np.abs(df['horsepower'] > 182.00)]= 182.00 # clippomg variables at 96 percentile
df['horsepower'][np.abs(df['horsepower'] > 3657.80)]= 3657.80 # clippomg variables at 96 percentile
df['enginesize'][np.abs(df['enginesize'] > 209.00)]= 209.00 # clippomg variables at  96 percentile
df['compressionratio'][np.abs(df['compressionratio'] > 10.94)]= 10.94 # clippomg variables at 90 percentile

### boxplot for all categorical values

In [ ]:
categorical_values=['Symboling','carCompany','fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','enginetype','cylindernumber','fuelsystem']

In [ ]:
#plotting all categorical variables to see relation with price
plt.figure(figsize=(15,40))
plt.subplot(10,1,1)
sns.boxplot(x='fueltype',y='price',data=df)
plt.subplot(10,1,2)
sns.boxplot(x='aspiration',y='price',data=df)
plt.subplot(10,1,3)
sns.boxplot(x='doornumber',y='price',data=df)
plt.subplot(10,1,4)
sns.boxplot(x='carbody',y='price',data=df)
plt.subplot(10,1,5)
sns.boxplot(x='drivewheel',y='price',data=df)
plt.subplot(10,1,6)
sns.boxplot(x='enginelocation',y='price',data=df)
plt.subplot(10,1,7)
sns.boxplot(x='enginetype',y='price',data=df)
plt.subplot(10,1,8)
sns.boxplot(x='cylindernumber',y='price',data=df)
plt.subplot(10,1,9)
sns.boxplot(x='fuelsystem',y='price',data=df)
plt.subplot(10,1,10)
sns.boxplot(x='symboling',y='price',data=df)
plt.show()

Observations:
    
Although not significant but still the fuel type seems to have an effect on the pricing of the cars. <br>
Enginelocation (rear) and aspiration(turbo) has a visible effect on the pricing of the car.<br>
The price of real wheel drive is significantly higher that other drivewheel options.<br>
Cylindernumber and engine type also seem to regulate the price of cars.<br>
Hardtop and convertables cars are priced higher than other body types available. <br>

In [ ]:
#plotting Carcompany with respect to price
plt.figure(figsize=(20,10))
sns.boxplot(x='CarName',y='price',data=df)
plt.xticks(rotation=90)
plt.show()

mean price for top companies:<br>
jaguar    :      34600.00<br>
buick     :     33647.00<br>
porsche   :     31400.50<br>
bmw       :    26118.75<br>
volvo     :     18063.18<br>

# 3. Data preparation


### Creating new variables(Derived metrics)
This will be helpful to remove correlated variables.<br>

Its observed that there is a high correlation between carlength, wheelbase, car width, car weight and city/highway mpg's.<br>
Lets create new variables from these to try reducing the multicolinearlity.<br>

In [ ]:
# Creating new variable carLWratio
df['carLWratio'] = df.carlength/df.carwidth
# Creating new variable carWHratio
df['carWHratio'] = df.carwidth/df.carheight
# Creating new variable PWratio
df['PWratio'] = df.horsepower/df.curbweight
# Creating new variable HCmpgratio
df['HCmpgratio'] = df.highwaympg/df.citympg
## droping the orignal variables
df.drop(['carlength','carwidth','carheight','highwaympg','citympg'],axis=1,inplace=True)

Since we saw that the company brand value is determinig the pricing of the car.
So We will segment the car companies based on the mean company price as:

lower if company mean price is below 10,000<br>
mid if company mean price is above 10,000 and below 20,000<br>
higher if company mean price is above 20,000<br>

In [ ]:
#finding mean price with respect to car company name
df.groupby(['CarName']).price.mean().sort_values(ascending=False)

In [ ]:
#segment car comapnies based on mean price range
company_segment_dict = {
    'cheverolet' : 'lower',
    'dodge' : 'lower',
    'plymouth' : 'lower',
    'honda' : 'lower',
    'subaru' : 'lower',
    'isuzu' : 'lower',
    'mitsubishi' : 'lower',
    'renault' : 'lower',
    'toyota' : 'lower',
    'volkswagen' : 'mid',
    'nissan' : 'mid',
    'mazda' : 'mid',
    'saab' : 'mid',
    'peugeot' : 'mid',
    'alfa-romero' : 'mid',
    'mercury' : 'mid',
    'audi' : 'mid',
    'volvo' : 'mid',
    'bmw' : 'higher',
    'buick' : 'higher',
    'porsche' : 'higher',
    'jaguar' : 'higher',
    }
df['company_segment'] = df['CarName'].map(company_segment_dict)
# Dropping the orignal carName variable
df.drop('CarName',axis=1,inplace=True)
df.head()

In [ ]:
df.groupby('company_segment').price.mean()

### creating dummy variables for all categorical variables

In [ ]:
# unique values for fueltype
print("unique values in fueltype")
print(df['fueltype'].unique())
print('\n')

# unique values for aspiration
print("Unique values in aspiration")
print(df['aspiration'].unique())

print('\n')

# unique values for doornumber
print("Unique values in doornumber")
print(df['doornumber'].unique())

print('\n')

# unique values for carbody
print("Unique values in carbody")
print(df['carbody'].unique())

print('\n')

# unique values for drivewheel
print("Unique values in drivewheel")
print(df['drivewheel'].unique())

print('\n')

# unique values for enginelocation
print("Unique values in enginelocation")
print(df['enginelocation'].unique())

print('\n')

# unique values for enginetype
print("Unique values in enginetype")
print(df['enginetype'].unique())

print('\n')

# unique values for cylindernumber
print("Unique values in cylindernumber")
print(df['cylindernumber'].unique())

print('\n')

# unique values for fuelsystem
print("Unique values in fuelsystem")
print(df['fuelsystem'].unique())

In [ ]:
#creating dummy variables
status=pd.get_dummies(df[['company_segment','carbody','drivewheel','enginetype','cylindernumber','fuelsystem','fueltype','aspiration','doornumber','enginelocation','symboling']],drop_first=True)
status.head()

In [ ]:
df=pd.concat([df,status],axis=1) #adding all dummy variavles with original dataset
df.head()

In [ ]:
#dropping variables whose dummy has already been created as those colums will not be required
df=df.drop(['company_segment','carbody','drivewheel','enginetype','cylindernumber','fuelsystem','fueltype','aspiration','doornumber','enginelocation','symboling'],axis=1)
df.head()

In [ ]:
df.shape

# 4. splitting the data into training and test set

#### the first basic step for regression is performing a train-test split.

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(0)
df_train,df_test=train_test_split(df,train_size=0.7,test_size=0.3,random_state=100)

###### Rescaling the Features using min_max scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler=MinMaxScaler()

In [ ]:
varlist=['price','peakrpm','horsepower','compressionratio','stroke','boreratio','enginesize','curbweight','wheelbase','carLWratio','carWHratio','PWratio','HCmpgratio']

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
df_train[varlist]=scaler.fit_transform(df_train[varlist])

In [ ]:
df_train.head()

In [ ]:
df_train.describe()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated
df_train.corr()

In [ ]:
# from above correlation enginesize seems to be correlated to price the most. Let's see a pairplot for area vs price.
plt.figure(figsize=[6,6])
plt.scatter(df_train.enginesize, df_train.price)
plt.show()

### Dividing into x and y sets for the model building

In [ ]:
y_train=df_train.pop('price')
x_train=df_train

#  5. Building a linear model

In [ ]:
# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [ ]:
#Running RFE with output numnber of varibales 15
lm=LinearRegression()
lm.fit(x_train,y_train)
rfe=RFE(lm,15)
rfe=rfe.fit(x_train,y_train)

In [ ]:
list(zip(x_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
col=x_train.columns[rfe.support_]
col

In [ ]:
x_train.columns[~rfe.support_]

### Building model using statsmodel, for the detailed statistics

In [ ]:
x_train_rfe=x_train[col]
x_train_rfe.head()

In [ ]:
import statsmodels.api as sm

Fit a regression line through the training data using `statsmodels`.
In `statsmodels`, we need to explicitly fit a constant using `sm.add_constant(X)` because if we don't perform this step, `statsmodels` fits a regression line passing through the origin, by default.

In [ ]:
# Add a constant
x_train_rfe=sm.add_constant(x_train_rfe)

In [ ]:
# Create a first fitted model
lr=sm.OLS(y_train,x_train_rfe).fit()

In [ ]:
print(lr.summary())

In [ ]:
# Check for the VIF values of the feature variables.
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_rfe.columns
vif['VIF']=[variance_inflation_factor(x_train_rfe.values,i)for i in range(x_train_rfe.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train_new=x_train_rfe.drop(['cylindernumber_twelve'],axis=1) # dropping enginetype_rotor because it has very high p value

In [ ]:
import statsmodels.api as sm 
x_train_new=sm.add_constant(x_train_new)
# Create a second fitted model
lr1=sm.OLS(y_train,x_train_new).fit()
print(lr1.summary())

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_new.columns
vif['VIF']=[variance_inflation_factor(x_train_new.values,i)for i in range(x_train_new.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train_new1=x_train_new.drop(['carWHratio'],axis=1) #dropping because of high p value

In [ ]:
x_train_new1=sm.add_constant(x_train_new1)
# Create a third fitted model
lr2=sm.OLS(y_train,x_train_new1).fit()
print(lr2.summary())

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_new1.columns
vif['VIF']=[variance_inflation_factor(x_train_new1.values,i)for i in range(x_train_new1.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train_new2=x_train_new1.drop(['horsepower'],axis=1)#dropping beacuse of high VIF

In [ ]:
x_train_new2=sm.add_constant(x_train_new2)
# Create a fourth fitted model
lr3=sm.OLS(y_train,x_train_new2).fit()
print(lr3.summary())

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_new2.columns
vif['VIF']=[variance_inflation_factor(x_train_new2.values,i)for i in range(x_train_new2.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif

In [ ]:
x_train_new3=x_train_new2.drop(['carLWratio'],axis=1)#dropping because of high p value

In [ ]:
x_train_new3=sm.add_constant(x_train_new3)
# Create a fifth fitted model
lr4=sm.OLS(y_train,x_train_new3).fit()
print(lr4.summary())

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_new3.columns
vif['VIF']=[variance_inflation_factor(x_train_new3.values,i)for i in range(x_train_new3.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif                                                                                                                                                                       

In [ ]:
x_train_new4=x_train_new3.drop(['enginetype_dohcv'],axis=1)# dropping p value greater than 0.05

In [ ]:
x_train_new4=sm.add_constant(x_train_new4)
# Create a sixth fitted model
lr5=sm.OLS(y_train,x_train_new4).fit()
print(lr5.summary())

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif=pd.DataFrame()
vif['Features']=x_train_new4.columns
vif['VIF']=[variance_inflation_factor(x_train_new4.values,i)for i in range(x_train_new4.shape[1])]
vif['VIF']=round(vif['VIF'],2)
vif=vif.sort_values(by='VIF',ascending=False)
vif 

In [ ]:
x_train_new5=x_train_new4.drop(['const'],axis=1) #dropping constant

# 6. Residual Analysis of the train data

In [ ]:
y_train_price=lr5.predict(x_train_new4)


In [ ]:
plt.figure(figsize=(10,5))
sns.distplot((y_train-y_train_price),bins=20)
plt.suptitle('Error Terms', fontsize = 20)                  
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)
plt.show()


# 7. Making Predictions Using the Final Model

#### Applying the scaling on the test sets

In [ ]:
varlist=['price','peakrpm','horsepower','compressionratio','stroke','boreratio','enginesize','curbweight','wheelbase','carLWratio','carWHratio','PWratio','HCmpgratio']
df_test[varlist]=scaler.transform(df_test[varlist])

### Dividing into x_test and y_test

In [ ]:
y_test=df_test.pop('price')
x_test=df_test

In [ ]:
# Now let's use our model to make predictions.
# Creating X_test_new dataframe by dropping variables from X_test

import statsmodels.api as sm
x_test_new=x_test[x_train_new5.columns]

# Adding a constant variable 
x_test_new=sm.add_constant(x_test_new)

In [ ]:
# Making predictions
y_pred=lr5.predict(x_test_new)

# 8. Model Evaluation

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig=plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y pred vs y test',fontsize=20)
plt.xlabel('y_test',fontsize=20)
plt.ylabel('y_pred',fontsize=20)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
print('Mean_Squared_Error :' ,mse)
print('r_square_value :',r_squared)

### Final inference

#### We can see that the equation of our best fitted line is:

$ price = 0.259  \times  wheelbase + 0.66  \times  enginesize - 0.202 \times stroke + 0.127 \times PWratio + 0.082 \times enginetype_ohc - 0.1044 \times enginetype_ohcv - 0.229 \times cylindernumber_five - 0.29 \times cylindernumber_four+ 0.268 \times enginelocation_rear-0.243 \times cylindernumber_six $

With a low p-value and low VIF, below variables do describe the price of the automobiles to a good extent.<br>

wheelbase<br>
enginesize<br>
PW ratio<br>
enginetype_ohc<br>
enginelocation_rear<br>
cylindernumber_five<br>

R squared for train model is : 0.88 <br>
R squared for test model is : 0.86 <br>
since there is only difference of 0.02 between test and train model so overall it is a decent model.

To make the model even better we have a couple of options:<br>

1)Add new features <br>
2)choosing another set of variables to get a more normal distribution of error terms<br>
3)Build a non-linear model<br>